In [6]:
from bs4 import BeautifulSoup
from datetime import date,datetime, timedelta
import re, os, requests
import pandas as pd

In [7]:
head = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36 Edg/86.0.622.38'}

In [8]:
#뉴스기사 주소 수집
def naver_news(start_date = None, end_date = None):
    sid2_cate = [259, 258, 261, 771, 260, 310, 263]
    url = "https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2={}&sid1=101&date={}&page={}"
    head = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36 Edg/86.0.622.38'}
    
    for date_ in [str(x).replace("-", "")[:8] for x in pd.date_range(start_date, end_date)]: #pd.date_range : 기간 생성(workingday로 주말 뺄 수도 있음)
        for cate in sid2_cate:
            r= requests.get(url.format(cate, date_, 200), headers= head) #200페이지를 썼을 때, 오류가 발생하지 않고 맨 마지막 페이지로 감
            bs = BeautifulSoup(r.text, "lxml")
            #print(x , end=' : ')
            
            #최종 페이지를 들고옴
            last_page = bs.find("div", class_='paging').find("strong").text
            
            #print ("last page:{}".format(last_page)
            for page_num in range(1, int(last_page)+1):
                r2= requests.get(url.format(cate, date_, page_num ), headers= head)
                bs2 = BeautifulSoup(r2.text, 'lxml')
                
                for x in bs2.find("div", class_="list_body newsflash_body").findAll("dt"):
                    article(date_, x.a['href']) #dt에 있는 a태그의 'href'를 수집

In [9]:
#정규식을 사용해 필요없는 정보 삭제
def article(date_, url):
    reporter = re.compile("[가-힣]{2,4}\s*기자")
    email   = re.compile("[\w._%+-]+@[\w.-]+\.[a-zA-Z]{2,4}")
    r = requests.get(url, headers= head)
    bs = BeautifulSoup(r.text)
    try:
        rt = bs.find("div", id="articleBodyContents")
        text = rt.text.strip()
    except:
        return  
    
    try:
        text = text[:text.find(rt.select_one("a").text)]
    except:
        pass
    
    #정규식으로 email, reporter 삭제함 
    text = re.sub(email, "", text)
    text = re.sub(reporter, "", text)
    
    #폴더 생성
    if not os.path.isdir(date_):
        os.mkdir(date_)
    
    f = open(date_ + "/" + url.split("?")[-1] + ".txt", "w", encoding='utf-8')
    f.write(text)
    f.close()

In [ ]:
naver_news('2021-05-25', '2021-05-26')